# Run Pre-defined Problems

This notebook is run on Google Colab. You need to manually upload the data to kaggle and pass the correct path to the pre-processing function.

In [ ]:
!pip install tensorflow-gpu
!pip install bert-multitask-learning
!rm -r transformers
!git clone https://github.com/huggingface/transformers.git && cd transformers && pip install .

In [ ]:
import tensorflow as tf
import transformers
from bert_multitask_learning import train_bert_multitask, train_eval_input_fn, BertMultiTask, DynamicBatchSizeParams
from bert_multitask_learning.predefined_problems import get_weibo_ner_fn, get_weibo_cws_fn

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
problem_type_dict = {
    'weibo_cws': 'seq_tag',
    'weibo_ner': 'seq_tag'
}

# data 
processing_fn_dict = {
    'weibo_ner': get_weibo_ner_fn(file_path='../input/weibo-ner/weiboNER*'),
    'weibo_cws': get_weibo_cws_fn(file_path='../input/weibo-ner/weiboNER*')
}

## Train Models
If you don't want to control every thing, you can just call `train_bert_multitask` function. Please note that starting from 0.4.2, transformer models (the body model) are implemented using [huggingface transformers](https://github.com/huggingface/transformers) and because of that, now we can basically use all transformer models by specifying following params(below is the default value):

    params.transformer_model_name = 'bert-base-chinese'
    params.transformer_tokenizer_name = 'bert-base-chinese'
    params.transformer_config_name = 'bert-base-chinese'
    params.transformer_model_loading = 'TFAutoModel'
    params.transformer_config_loading = 'BertConfig'
    params.transformer_tokenizer_loading = 'AutoTokenizer'

  And for decoder:

    params.transformer_decoder_model_name = None
    params.transformer_decoder_config_name = None
    params.transformer_decoder_tokenizer_name = None
    params.transformer_decoder_model_loading = 'TFAutoModel'
    params.transformer_decoder_config_loading = 'BertConfig'
    params.transformer_decoder_tokenizer_loading = 'AutoTokenizer'

However, there are still some drawbacks when initializing Keras model within estimator. To overcome this, we need to manually download the weights and pass the model path to `params.transformer_model_name`.

In [ ]:
transformers.TFAutoModel.from_pretrained('bert-base-chinese').save_weights('./models/bert-base-chinese/pretrained_model')

In [ ]:
# here we use the default model which is bert-base-chinese
params = DynamicBatchSizeParams()
# AutoConfig cannot load from dict...
params.transformer_config_loading = 'BertConfig'
params.transformer_model_name = './models/bert-base-chinese/pretrained_model'
train_bert_multitask(problem='weibo_ner&weibo_cws', params=params, problem_type_dict=problem_type_dict, processing_fn_dict=processing_fn_dict, num_gpus=1, num_epochs=50)

If you want to take more control of the training process, you can use lower level api

In [ ]:
import tensorflow as tf
from tensorflow.estimator import Estimator

problem = 'weibo_ner&weibo_cws'
num_gpus = 1
bert_multitask_params = DynamicBatchSizeParams()
bert_multitask_params.transformer_model_name = './models/bert-base-chinese/pretrained_model'

for new_problem, new_problem_processing_fn in processing_fn_dict.items():
    print('Adding new problem {0}, problem type: {1}'.format(
        new_problem, problem_type_dict[new_problem]))
    bert_multitask_params.add_problem(
        problem_name=new_problem, problem_type=problem_type_dict[new_problem], processing_fn=new_problem_processing_fn)

# assign problem to params
bert_multitask_params.train_epoch = 1
bert_multitask_params.assign_problem(problem, gpu=1)

In [ ]:
# get model fn and create mirror strategy for distributed training
model = BertMultiTask(params=bert_multitask_params)
model_fn = model.get_model_fn()

dist_trategy = tf.distribute.MirroredStrategy()

run_config = tf.estimator.RunConfig(
    train_distribute=dist_trategy,
    eval_distribute=dist_trategy,
    log_step_count_steps=params.log_every_n_steps)

estimator = Estimator(
    model_fn,
    model_dir=params.ckpt_dir,
    params=params,
    config=run_config)


In [ ]:
# train
def train_input_fn(): return train_eval_input_fn(bert_multitask_params)
estimator.train(
    train_input_fn, max_steps=200)


## Evaluate and Predict

~~For NER and CWS, we need different evaluation logic.~~ Evaluation has bug and not fixed now.

In [ ]:
from bert_multitask_learning import predict_bert_multitask

In [ ]:
# predict
import numpy as np
from bert_multitask_learning.utils import get_or_make_label_encoder
predict_params = DynamicBatchSizeParams()
# get prediction generator
pred_prob = predict_bert_multitask(
    inputs=['中国和美国在打贸易战']*10, 
    problem='weibo_cws&weibo_ner', 
    params=predict_params,
    processing_fn_dict=processing_fn_dict,
    problem_type_dict=problem_type_dict)
# get label encoder
ner_label_encoder = get_or_make_label_encoder(params=predict_params, problem='weibo_ner', mode='predict', label_list=[])
cws_label_encoder = get_or_make_label_encoder(params=predict_params, problem='weibo_cws', mode='predict', label_list=[])

for prob in pred_prob:
    ner_pred = np.argmax(prob['weibo_ner'], axis = -1)
    print(ner_label_encoder.inverse_transform(ner_pred.tolist()))